<a href="https://colab.research.google.com/github/BossAyush07/Deep-Learning-Projects/blob/master/AMZN_CH_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('mydrive')

Mounted at mydrive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical

import os

In [ ]:
df = pd.read_csv('/content/mydrive/MyDrive/AMZN_ML_CHALLENGE/sample50')
df.head(15)

,Unnamed: 0,node,level_1,text
0,0,0,708239,disney princess jasmine fashion doll
1,1,0,2788330,mattel disney princess fairytale magiclip cind...
2,2,0,2375629,barbie princess doll assortment barbie prince...
3,3,0,1891769,mini lalaloopsy doll keys sharps n flatsproduc...
4,4,0,2665110,piece baby doll care accessories bagyou pie...
5,5,0,900829,little mommy sweet party shimmer baby doll mon...
6,6,0,245950,barbie designer collecton gold label anna sui ...
7,7,0,131048,barbie doll world landmark collection big ben ...
8,8,0,506733,alimrose aurelie linen cat doll
9,9,0,1683057,ever high bunny blanc doll


In [ ]:
df.isnull().sum()

Unnamed: 0    0
node          0
level_1       0
text          1
dtype: int64

In [ ]:
df = df.dropna(axis = 0, how ='any')

In [ ]:
df.shape

(268516, 4)

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.01, random_state=42)

In [ ]:
import pickle

with open('/content/mydrive/MyDrive/AMZN_ML_CHALLENGE/tokenizer (4).pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
with open('/content/mydrive/MyDrive/AMZN_ML_CHALLENGE/encoder.pkl', 'rb') as enc_file:
    enc = pickle.load(enc_file) 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
MAX_SEQUENCE_LENGTH = 100
batch_size = 256

In [ ]:
def batch_generator(x, y, batch_size=32):
    n_samples = len(x)
    i=0
    while True:
        x_batch, y_batch = [], []
        for b in range(batch_size):
            if i == n_samples:
                i = 0
            X = tokenizer.texts_to_sequences([' '.join(x[i].split())])
            
            X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
            Y = np.asarray(y[i])
            x_batch.append(np.array(X[0] , dtype = np.float))
            y_batch.append(Y)
            
            i+=1
        yield np.array(x_batch), enc.transform(np.array(y_batch).reshape(-1, 1)).toarray()

In [ ]:
train_gen = batch_generator(train_data['text'].values, train_data['node'].values, batch_size)
test_gen = batch_generator(test_data['text'].values, test_data['node'].values, batch_size)

### Downloading the glove pretrained embeddings

---



In [ ]:
#!wget https://nlp.stanford.edu/data/glove.840B.300d.zip

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2021-07-31 16:28:58--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-31 16:28:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.03MB/s    in 2m 40s  

2021-07-31 16:31:38 (5.15 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


### Loading the embedding into the memory

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


### embedding matrix by assigning the vocabulary with the pretrained word embeddings:

In [ ]:
max_features = 100000
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size) 

967294


In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model

MAX_SEQUENCE_LENGTH = 100
max_features = 100000
EMBEDDING_DIM = 100
N_CLASSES = 9919

# input: a sequence of MAX_SEQUENCE_LENGTH integers
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(vocab_size, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
embedded_sequences = embedding_layer(sequence_input)

average = GlobalAveragePooling1D()(embedded_sequences)
predictions = Dense(N_CLASSES, activation='softmax')(average)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['acc'])

In [ ]:
model.fit(train_gen, epochs=50, validation_data=test_gen, steps_per_epoch = len(train_data)//batch_size, validation_steps=len(test_data) // batch_size)

Epoch 1/50
1038/1038 [==============================] - 60s 44ms/step - loss: 8.9912 - acc: 0.0049 - val_loss: 8.6601 - val_acc: 0.0316
Epoch 2/50
1038/1038 [==============================] - 45s 43ms/step - loss: 8.5479 - acc: 0.0415 - val_loss: 8.3899 - val_acc: 0.0750
Epoch 3/50
1038/1038 [==============================] - 45s 43ms/step - loss: 8.2780 - acc: 0.0801 - val_loss: 8.1662 - val_acc: 0.0934
Epoch 4/50
1038/1038 [==============================] - 45s 44ms/step - loss: 8.0404 - acc: 0.1064 - val_loss: 7.9742 - val_acc: 0.1051
Epoch 5/50
1038/1038 [==============================] - 46s 44ms/step - loss: 7.8284 - acc: 0.1268 - val_loss: 7.7962 - val_acc: 0.1227
Epoch 6/50
1038/1038 [==============================] - 46s 44ms/step - loss: 7.6367 - acc: 0.1431 - val_loss: 7.6472 - val_acc: 0.1352
Epoch 7/50
1038/1038 [==============================] - 46s 44ms/step - loss: 7.4629 - acc: 0.1566 - val_loss: 7.4783 - val_acc: 0.1488
Epoch 8/50
1038/1038 [==========================

In [ ]:
model.fit(train_gen, epochs=25, validation_data=test_gen, steps_per_epoch = len(train_data)//batch_size, validation_steps=len(test_data) // batch_size)

Epoch 1/25
1038/1038 [==============================] - 46s 44ms/step - loss: 4.7857 - acc: 0.3421 - val_loss: 5.2618 - val_acc: 0.2676
Epoch 2/25
1038/1038 [==============================] - 44s 43ms/step - loss: 4.7615 - acc: 0.3440 - val_loss: 5.3079 - val_acc: 0.2648
Epoch 3/25
1038/1038 [==============================] - 46s 44ms/step - loss: 4.7376 - acc: 0.3459 - val_loss: 5.2683 - val_acc: 0.2684
Epoch 4/25
1038/1038 [==============================] - 44s 42ms/step - loss: 4.7145 - acc: 0.3478 - val_loss: 5.2719 - val_acc: 0.2652
Epoch 5/25
1038/1038 [==============================] - 45s 43ms/step - loss: 4.6918 - acc: 0.3497 - val_loss: 5.2193 - val_acc: 0.2688
Epoch 6/25
1038/1038 [==============================] - 45s 43ms/step - loss: 4.6694 - acc: 0.3515 - val_loss: 5.2146 - val_acc: 0.2695
Epoch 7/25
1038/1038 [==============================] - 46s 44ms/step - loss: 4.6478 - acc: 0.3534 - val_loss: 5.1998 - val_acc: 0.2738
Epoch 8/25
1038/1038 [==========================

In [ ]:
model.fit(train_gen, epochs=30, validation_data=test_gen, steps_per_epoch = len(train_data)//batch_size, validation_steps=len(test_data) // batch_size)

Epoch 1/30
1038/1038 [==============================] - 44s 43ms/step - loss: 4.3065 - acc: 0.3842 - val_loss: 4.9621 - val_acc: 0.2855
Epoch 2/30
1038/1038 [==============================] - 45s 43ms/step - loss: 4.2915 - acc: 0.3856 - val_loss: 4.9580 - val_acc: 0.2902
Epoch 3/30
1038/1038 [==============================] - 45s 43ms/step - loss: 4.2768 - acc: 0.3868 - val_loss: 4.9524 - val_acc: 0.2898
Epoch 4/30
1038/1038 [==============================] - 45s 43ms/step - loss: 4.2621 - acc: 0.3882 - val_loss: 4.9361 - val_acc: 0.2914
Epoch 5/30
1038/1038 [==============================] - 44s 42ms/step - loss: 4.2481 - acc: 0.3895 - val_loss: 4.9447 - val_acc: 0.2883
Epoch 6/30
1038/1038 [==============================] - 45s 43ms/step - loss: 4.2339 - acc: 0.3906 - val_loss: 4.9218 - val_acc: 0.2922
Epoch 7/30
1038/1038 [==============================] - 44s 43ms/step - loss: 4.2201 - acc: 0.3920 - val_loss: 4.9410 - val_acc: 0.2855
Epoch 8/30
1038/1038 [==========================

### Setting up for predictions

In [ ]:
 test_df = pd.read_csv('/content/mydrive/MyDrive/AMZN_ML_CHALLENGE/prepared_test.csv')

In [ ]:
x_batch, y_batch = [], []
for i in range(len(test_df['text'].values)):
    X = tokenizer.texts_to_sequences([test_df['text'].values[i]])
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    x_batch.append(np.array(X[0] , dtype = np.float))

X = np.array(x_batch)

In [ ]:
predictions = []

In [ ]:
for i in range(0, len(X), 100):

    if i%1000==0:
        print(i)
    y_pp = model.predict(X[i:i+100])
    y_p = list(enc.inverse_transform(y_pp).reshape(-1))
    predictions += y_p

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000


In [ ]:
sub = pd.DataFrame()
sub['PRODUCT_ID'] = list(range(len(predictions)))
sub['BROWSE_NODE_ID'] = predictions

In [ ]:
sub.to_csv('submission.csv', index=False)